<a href="https://colab.research.google.com/github/Kshitij04Poojary/Iterated-Prisoners-Dilemma/blob/main/IPDRPexpl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install optuna
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
import optuna

Declaring Hyperparameters globally


In [ ]:
input_size = 5
output_size = 2 #no of actions an agent can take
epsilon_start = 1.0 #initial value of epsilon
epsilon_end = 0.01 #final value of epsilon
gamma = 0.99 #discount factor for rewards
num_episodes = 200

The following is a simple LSTM. This is what we use to create the DQN Model. Additionally, the output of this is the q values for each action (this needs to be raw q values, hence we dont have to use a softmax or any sort of activation function)



In [ ]:
class DQN(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, layer_num):
        super().__init__()
        self.lstmLayer = nn.LSTM(in_dim, hidden_dim, layer_num, batch_first=True) #enabling batch_first ensures that the shape of tensors is maintained as [batch_size,sequence_length,layer_num]
        self.relu = nn.ReLU() #applies RELU( Rectified Linear Unit) activation function introducing non linearity (model can learn more complex things)
        self.fcLayer = nn.Linear(hidden_dim, out_dim) #fully connected layer to map from hidden to o/p state

    def forward(self, x): #function to pass the data through the network layers where the input tensor is of shape [batch_size, seq_len, hidden_num]
        out, _ = self.lstmLayer(x) #output features for each time step is stored
        out = self.relu(out[:, -1, :])  # filtered to take o/p features of last time step only [shape of out changes from [batch_size, seq_len, layer_num] to [batch_size, hidden_num]]
        out = self.fcLayer(out) # passes out to a fully connected layer making the dimension of output tensor [[batch_size, out_size]]
        return out

Replay Buffer is basically a Circular Stack (Dequeue?). It is a data structure used to track and access transitions that the agent encounters. The 4 states are : state (current state), action (action taken by the agent according to the current state), reward (reward recieved immeadeatly by the agent for the action), next_state(state observed after the current action). The whole point of the replay buffer is that it allows the agent to learn from it's previous actions. In it's sample function, it picks up random transitions according to the batch size. This is then used to update the parameters of the agent's Neural Network. It is random to minimize the bias it can form due to recent experiences and allowing the agent to learn from diverse experiences.

In [ ]:
class ReplayBuffer:
  def __init__(self,cap):
    self.buffer = []
    self.cap = cap      #cap is the capacity of the buffer
    self.pos = 0
  def push(self,state,action,reward,next_state):  #push function to append a new transition in the buffer
    if len(self.buffer) < len(self.cap):
      self.buffer.append(None)          #initialize the buffer with None
    self.buffer[self.pos] = (state,action,reward,next_state)
    self.pos = (self.pos + 1) % self.cap
  def sample(self,batch):             #zips random batch of transitions according to the batch size and returns it
    return zip(*random.sample(self.buffer,batch))
  def len(self):
    return len(self.buffer)


The payoff matrix basically aims to maximize the profit, which in a non-iterated prisoner's dilemma is the one where 1 deflects and the other coorporates. The rewards are assigned with this in mind. Rows indicate the agent(player1) and columns the oppnent(player2).


Both Coorporate : Agent - 3, Opp - 0

1 Defect, 1 Coorporate : Agent - 5, Opp-1 (Agent Defects)

1 Coorporate , 1 Defect : Agent - 1 , Opp - 5 (Opp Defects)

Both Defect : Agent - 0, Opp - 0

In [ ]:
class IPD:
  def __init__(self):
    self.actions = 2 #Coorporate or Deflect
    self.payoff = np.array([[3, 0], [5, 1], [1, 5], [0, 0]])

  def step(self,action1,action2):
    reward1 = self.payoff[action1][action2]
    reward2 = self.payoff[action2][action1]
    return reward1,reward2

The below segment is a part of the DQN algorithm. Epsilon is the "exploration" parameter. In the below section, we check if the random number is less than epsilon, in which case we will choose exploration and pick a random action. Else, we choose to exploit the network's previous knowledge. This is done by calculating all q values of the policy network and picking the action with the highest Q Value.

After unsqueeze operation 2 times the shape of the tensor becomes [1, 1, input_size]. because we process single step(seq_len) of a single batch(batch_size) at a time.

In [ ]:
def select_action(state,epsilon):
  if np.random.rand() < epsilon:  #exploration
    return np.random.randint(output_size) #pick random action
  else:
    with torch.no_grad():   #exploitation
      state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0).unsqueeze(0) #calculates q values. converts state into a tensor. (torch no grad makes computation faster)
      q_values = policy_net(state_tensor)
      return q_values.argmax().item()#finds maximum q value and finds the scalar value of the index of the action which is a tensor and return it as action

The below step is used to update the q values in the policy network itself (The target network is then later updated after some amt of time). We sample a mini batch from the replay buffer and then convert it to tensors. We then calculate the q_values(current state), next_q_values(next state) and expected q values using the Bellman Equation. We than calculate the loss with respect to the q values of actions actually taken by the agent and the expected q values. Loss is calculated by gradient back propogation and then the parameters that lead to the least amount of loss is updated in the policy network.

In [ ]:
def update_q_values(replay_buffer, batch_size, policy_net, target_net, optimizer, gamma):
  if len(replay_buffer) < batch_size:  #checks if there are enough transitions in the replay buffer to make a mini batch
    return
  state, action, reward, next_state = replay_buffer.sample(batch_size)
  state = torch.tensor (state, dtype = torch.float32).view(batch_size, 1, -1)  #convert state to tensor and reshape it
  action = torch.tensor (action, dtype = torch.long)   #actions are convert into long tensor because actions are integers and can either be 0 or 1
  reward = torch.tensor (reward, dtype = torch.float32)
  next_state = torch.tensor (next_state, dtype = torch.float32).view(batch_size, 1, -1) # use view to reshape it

  q_values = policy_net(states).gather(1, actions.unsqueeze(1)).squeeze(1)   #calculate q values of current state, adds a column tensor to reshape and then change it back to og dimension of(batch_size,)
  next_q_values = target_network(next_state).max(1)[0].detach()    #calculate q values of next state and finding its maximum
  expected_q_values = reward + gamma* next_q_vals   #Bellman's Equation to find expected q values
  loss = nn.functional.mse_loss(q_values, expected_q_values)  #calculates loss b/w target q vals and predicted q vals
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()   #updates params of policy network that give the least amount of loss

Below is the main training loop.
Initial steps of the DQN algorithm involves making a policy network and a target network that are one and the same. The target network is a replica of the policy network

epsilon = epsilon_end + (epsilon_start-epsilon_end)*np.exp(-episode/epsilon_decay).

In the above formula. epsilon_end is the value of epsilon that is tending to zero. This is the target value of epsilon which represents the minimum exploration rate. epsilon_start is the value of epsilon that is closer to 1. It is the initial value of epsilon which represents the maximum exploration rate. epsilon_decay is the factor which is responsible for the decrease of epsilon over time, so that the Network can move from exploration of new values to exploiting its previous knowledge.

np.exp(-episode/epsilon_decay).This is known as the exponential decay factor which is negative and progressively becomes more negative as episode number increases. This causes the epsilon value to tend to zero over time.
(epsilon_start-epsilon_end)*np.exp(-episode/epsilon_decay).This essentially scales the value between epsilon start and epsilon end. Multiplying it with the exponential decay factor makes the epsilon value shift from epsilon start to epsilon end.
epsilon_end + (epsilon_start-epsilon_end)*np.exp(-episode/epsilon_decay).Adding it to the epsilon_end makes it so that it allows the value of epsilon to decrease over time while also allowing exploration in the early stages of training.

Overview of HyperParam Tuning:

Optuna - Open Source FrameWork for hyperparam tuning. Uses Beysian Probab Approach to find the optimal params (probabilistic model of the obj func is built and then we minimize/maximise it)

The study is an object that manages the optimization of the obj function. we have set it to maximise the obj function

Suggestion Methods to define search range : suggest_categorical (pick one from list), uniform/loguniform/int(suggest a value uniformly/log value uniformly/integer value in the given range)

In [ ]:
def train_dqn(trial):
    # Hyperparameters to optimize
    hidden_size = trial.suggest_categorical('hidden_size', [32, 64, 128])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    epsilon_decay = trial.suggest_uniform('epsilon_decay', 0.99, 0.999)
    target_update = trial.suggest_int('target_update', 5, 20)

    policy_net = DQN(input_size, hidden_size, output_size, 1) #this network makes decisions
    target_net = DQN(input_size, hidden_size, output_size, 1) #this network stabilizes the training processs
    target_net.load_state_dict(policy_net.state_dict()) #copy wts from policy to target network
    target_net.eval() #eval mode

    optimizer = optim.Adam(policy_net.parameters(), lr=learning_rate) # The learning rate is a hyperparameter that controls how much the optimizer adjusts the parameters with respect to the gradient. A higher learning rate means larger updates,faster learning process,
    #but it might overshoot the optimal solution. Conversely, a lower learning rate means smaller updates, which could make the learning process slower but more precise.
    replay_buffer = ReplayBuffer(capacity=10000)
    ipd_env = IPD()
    total_rewards = []

    for episode in range(num_episodes):
        state = [0, 0, 0, 0, 0] #initialization
        total_reward = 0
        for t in range(100):
            epsilon = epsilon_end + (epsilon_start - epsilon_end) * np.exp(-episode / epsilon_decay)
            action = select_action(state, epsilon, policy_net, output_size) #selects agents current action
            opponent_action = state[0] #opp ACTION TIT FOR TAT
            reward, opponent_reward = ipd_env.step(action, opponent_action)
            next_state = [action, opponent_action, reward, opponent_reward, t] # t is time step
            replay_buffer.push(state, action, reward, next_state)
            state = next_state
            total_reward += reward
            update_q_values(replay_buffer, batch_size, policy_net, target_net, optimizer, gamma)
            if t % target_update == 0:
                target_net.load_state_dict(policy_net.state_dict())#load policy network params to the target network periodically. This is determined by the target_update param
      #we defined and is called synchronization
        total_rewards.append(total_reward)

    avg_reward = np.mean(total_rewards)
    return avg_reward

study = optuna.create_study(direction='maximize') #creates an env that facilitates the hyperparameter tuning and aims to maximise the objective function
study.optimize(train_dqn, n_trials=50) #objective function to optimize is train_dqn with 50 trails.

print(f"Best trial: {study.best_trial.params}")

Use the best/most rewarding hyperparams to train the final model!!

In [ ]:
best_params = study.best_trial.params #extract best hyperparams
hidden_size = best_params['hidden_size']
learning_rate = best_params['learning_rate']
batch_size = best_params['batch_size']
epsilon_decay = best_params['epsilon_decay']
target_update = best_params['target_update']
#rest is just a repeat of the training process
policy_net = DQN(input_size, hidden_size, output_size, 1)
target_net = DQN(input_size, hidden_size, output_size, 1)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.Adam(policy_net.parameters(), lr=learning_rate)
replay_buffer = ReplayBuffer(capacity=10000)
ipd_env = IteratedPrisonersDilemma()

for episode in range(num_episodes):
    state = [0, 0, 0, 0, 0]
    total_reward = 0
    for t in range(100):
        epsilon = epsilon_end + (epsilon_start - epsilon_end) * np.exp(-episode / epsilon_decay)
        action = select_action(state, epsilon, policy_net, output_size)
        opponent_action = state[0]
        reward, opponent_reward = ipd_env.step(action, opponent_action)
        next_state = [action, opponent_action, reward, opponent_reward, t]
        replay_buffer.push(state, action, reward, next_state)
        state = next_state
        total_reward += reward
        update_q_values(replay_buffer, batch_size, policy_net, target_net, optimizer, gamma)
        if t % target_update == 0:
            target_net.load_state_dict(policy_net.state_dict())
    print(f"Episode {episode + 1}, Total Reward: {total_reward}")

Simulate the IPD game. This function helps us simulate various stratergies to test out our network





In [ ]:
def play_ipd(policy_net, num_episodes, strategy, epsilon=0.1):
    ipd_env = IPD()
    total_rewards = []
    for episode in range(num_episodes):
        state = [0, 0, 0, 0, 0]
        total_reward = 0
        opponent_last_action = 0
        grim_trigger_active = False
        for t in range(100):
            with torch.no_grad():
                if np.random.rand() < epsilon: #action selection stratergy
                    action = np.random.randint(output_size)
                else:
                    state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
                    q_values = policy_net(state_tensor)
                    action = q_values.argmax().item()

            if strategy == "random":
                opponent_action = np.random.randint(2)
            elif strategy == "tft":
                opponent_action = opponent_last_action
            elif strategy == "grim_trigger":
                if grim_trigger_active:
                    opponent_action = 1
                else:
                    opponent_action = 0
                    if action == 1:
                        grim_trigger_active = True
            elif strategy == "always_cooperate":
                opponent_action = 0
            elif strategy == "always_defect":
                opponent_action = 1

            reward, _ = ipd_env.step(action, opponent_action)
            total_reward += reward

            next_state = [action, opponent_action, reward, 0, t]
            state = next_state
            opponent_last_action = action

        total_rewards.append(total_reward)
        print(f"Episode {episode + 1}, Total Reward: {total_reward}")

    return total_rewards


TESTING AGAINST SOME COMMON STRATERGIES!!

In [ ]:
#Tit For Tat Stratergy
def test_against_tft(policy_net, num_episodes):
    tft_rewards = play_ipd(policy_net, num_episodes, strategy="tft", epsilon=0.1)
    avg_reward = np.mean(tft_rewards)
    print("Average reward against TFT strategy:", avg_reward)

test_against_tft(policy_net, num_episodes=100)

In [ ]:
# Grim Trigger strategy
def test_against_grim(policy_net, num_episodes):
    grim_rewards = play_ipd(policy_net, num_episodes, strategy="grim_trigger")
    avg_reward = np.mean(grim_rewards)
    print(f"Average reward against Grim Trigger strategy: {avg_reward}")

test_against_grim(policy_net, num_episodes=100)

In [ ]:
# Always Cooperate strategy
def test_against_ac(policy_net, num_episodes):
    ac_rewards = play_ipd(policy_net, num_episodes, strategy="always_cooperate")
    avg_reward = np.mean(ac_rewards)
    print(f"Average reward against Always cooperate strategy: {avg_reward}")

test_against_ac(policy_net, num_episodes=100)

In [ ]:
# Always Defect strategy
def test_against_ad(policy_net, num_episodes):
    ad_rewards = play_ipd(policy_net, num_episodes, strategy="always_defect")
    avg_reward = np.mean(ad_rewards)
    print(f"Average reward against Always defect strategy: {avg_reward}")

test_against_ad(policy_net, num_episodes=100)

In [ ]:
# random stratergy
def test_against_random(policy_net, num_episodes):
    random_rewards = play_ipd(policy_net, num_episodes,strategy='random')
    avg_reward = np.mean(random_rewards)
    print("Average reward against random strategy:", avg_reward)


test_against_random(policy_net, num_episodes=100)


https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html

https://www.youtube.com/watch?v=t3fbETsIBCY


